In [43]:
from langchain_openai import ChatOpenAI
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
import os
import time

In [ ]:
load_dotenv()

# Components

In [2]:
llm = ChatOpenAI(
    openai_api_base="https://api.xiaoai.plus/v1",
    openai_api_key= os.getenv('OPENAI_API_KEY'),
    temperature=0.3
)
res = llm.invoke("What is after Monday?")
print(res.content)

After Monday comes Tuesday.


In [4]:
messages = [
    SystemMessage(content = "You are an expert data scientist"),
    HumanMessage(content = "Write a very simple python script that trains a neural network on simulated data")
]
response = llm(messages)
print(response.content)

Sure! Here's a simple example of a Python script that uses the popular library TensorFlow to train a neural network on simulated data:

```python
import numpy as np
import tensorflow as tf

# Generate simulated data
np.random.seed(0)
X = np.random.rand(100, 1)
y = 2 * X + 1 + 0.1 * np.random.randn(100, 1)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))
])

# Compile the model
model.compile(optimizer='sgd', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100)

# Make predictions
predictions = model.predict(X)

# Print the predictions
print(predictions)
```

In this script, we first generate some simulated data `X` and `y`. Then, we define a simple neural network model with one dense layer using `tf.keras.Sequential`. We compile the model using stochastic gradient descent optimizer and mean squared error loss. We then train the model on the simulated data for 100 epochs using `model.fit`. Finally, we mak

# Prompts

In [7]:
prompt = PromptTemplate(
    input_variables = ['topic'],
    template = """Please tell me a joke about {topic} with 50 words."""
)
response = llm(prompt.format(topic="Irish"))
print(response.content)

Why don't Irish people play hide and seek?

Because good luck trying to hide when everyone knows your name and where you live!

Why did the Irishman bring a ladder to the bar?

Because he heard the drinks were on the house!


# Chain

In [8]:
chain_one = prompt | llm | StrOutputParser()
response1 = chain_one.invoke({"topic": "Irish"})
print(response1)

Why don't Irish people play hide and seek?

Because good luck trying to hide when everyone knows your name, your family, and your entire life story after just one conversation at the pub!

But seriously, they're just too busy having the craic and making new friends to bother with hiding.


In [9]:
prompt_second = PromptTemplate(
    input_variables = ['joke'],
    template = """Is this {joke} funny?"""
)
overall_chain = {"joke": chain_one} | prompt_second | llm | StrOutputParser()
response2 = overall_chain.invoke({"topic": "Irish"})
print(response2)

It's a lighthearted and humorous take on the stereotype of Irish people being very social and interconnected. Some people may find it funny, while others may not. Humor is subjective!


# Embedding & Vector

In [13]:
prompt3 = PromptTemplate(
    input_variables = ["story"],
    template="""please make up a beautiful story based on topic of {story} within 300 words"""
)

story_example = llm(prompt3.format(story = 'princess'))
print(story_example.content)

Once upon a time in a faraway kingdom, there lived a beautiful and kind-hearted princess named Isabella. She was loved by all the people in the kingdom for her generosity and compassion towards everyone.

One day, a terrible curse fell upon the kingdom, and the land was plunged into darkness and despair. The people were suffering, and the king and queen were desperate to find a solution. They consulted the wise old wizard, who told them that the only way to break the curse was for the princess to embark on a perilous journey to find the magical crystal that held the power to restore peace and prosperity to the kingdom.

Without hesitation, Princess Isabella set out on her quest, accompanied by her loyal and brave companions. Along the way, they encountered many challenges and obstacles, but the princess never lost hope. She showed great courage and determination, and her unwavering spirit inspired everyone around her.

After a long and arduous journey, Princess Isabella finally found t

In [15]:
#text splitter
test_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0
)

texts = test_splitter.create_documents([story_example.content])
texts[0].page_content

In [24]:
#embedding
embeddings = OpenAIEmbeddings(
    openai_api_base="https://api.xiaoai.plus/v1",
    openai_api_key= os.getenv('OPENAI_API_KEY'),
)
query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

[3.1966247e-05, -0.020432582, -0.019580688, -0.015772972, -0.002286242, 0.0379997, -8.021814e-05, -0.02018734, 0.010184023, -0.01666359, 0.0058438736, -0.0036721856, 0.013126934, -0.012184686, -0.0054760096, 0.00090836576, 0.020109896, 0.01697337, 0.017631654, -0.0028880546, -0.0009002986, -0.007357279, -0.0042982, 0.0033398173, -0.0032123558, -0.010229199, 0.029997045, -0.035082605, 0.0055663623, -0.0027944753, -0.0063117705, -0.019245092, 0.0019264454, -0.010939112, -0.046467025, -0.027725324, -0.015308303, -0.00947411, -0.008435056, -0.014611297, 0.0083511565, 0.009816159, 0.0019425798, -0.007744504, -0.010764861, 0.011920082, -0.0074411775, 0.01941289, -0.02147809, 0.016160198, 0.0008285006, 0.02707995, -0.038954854, -0.005417926, -0.03017775, 0.002771887, 0.004507947, 0.012320215, -0.003025197, 0.008151091, 0.011016557, 0.014069182, -0.015024338, -0.007099129, -0.027492989, -0.002834811, -0.01951615, -0.006273048, 0.0021652342, -0.0050145662, 0.04182032, 0.011590941, 0.016276365, 

In [31]:
#pinecone
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = 'langchain-quickstart-today'

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
vector_store.add_documents(documents=texts)

['3b29ee38-a7a7-41ac-a04e-40de21551a55',
 'edc1a9bd-1d06-4c7c-9054-db625487f915',
 '2ffc0907-d728-49d0-9af4-c145cafc3912',
 '79587f62-185e-4ceb-8305-502ad456097d',
 '12c2ab91-49b6-451c-8608-c0426b2f2f51',
 '10c2ac66-38a5-438d-9dc5-6a52bcbd0751',
 'a7d3006e-c4e5-4a6c-9128-7449ab2e517d',
 'c5366e4a-5349-421c-a4e9-50c77b59ec57',
 'f5f3dca7-50da-4221-853c-5dbfa9f18351',
 '0720b4ac-d78a-4a05-92d7-d03d37ab3c0c',
 '53675963-f677-4957-aaea-299b7a9e8e61',
 '49bf88ff-bc33-4f78-9f8f-d1ba6ef853fe',
 'd4f777cc-a1a3-40c2-9866-f24c048019d2',
 '94171e5a-a54b-4fa8-9f69-fa3186520cde',
 '6a1f4d33-1226-4a37-84f5-5b0cd3d112cf',
 '179b1d43-363e-4512-8733-dd3788dba05b',
 '46ea9e40-9d44-4f2d-9289-f77aed6907cc',
 'a02ddaa2-531f-4e33-99ce-c72423f16c38',
 'cd6b9d28-bc85-41fc-a54f-7b7630af4980',
 'fefe36af-0700-46b5-90af-e8be3ef068e6',
 '06bc6d62-9311-40f7-8ffd-2d9758a56efc',
 '3c1917d8-3ea0-459a-9843-29c18488fccc',
 'cca73edb-2ffa-42dd-80ae-be639a2e111c',
 '3cfa4a7f-6fe7-41ba-84c5-e8ad0779cb6d']

In [40]:
query = "what happend in that kingdom?"
results = vector_store.similarity_search_with_score(query=query, k=2)
for res in results:
    print(res)

(Document(page_content='peace and prosperity to the kingdom.'), 0.848613918)
(Document(page_content='the cave, and the curse was lifted from the kingdom. The land was once again bathed in sunshine,'), 0.834380031)


# Agents